## Examine output of nnU-Net GTV H+N Segmentations

# Imports

In [ ]:
import JupyterNotebooksLib as slicernb
import SegmentStatistics
from ipywidgets import HBox
import pandas as pd
import csv
import time

# Load in Data File

In [ ]:
df = pd.read_csv('../nnU-NetResultsTrain.csv')
df = df.sort_values('Dice')

In [ ]:
# set up arrays
ctIntens_gt = {}
ptIntens_gt = {}
vol_gt = {}

ctIntens_pred = {}
ptIntens_pred = {}
vol_pred = {}

# Generate images for each patient, sorted by Dice

In [ ]:
i = 0 
for iterNum, row in df.iterrows():
    
#     if i >=1:
#         break

    # close scene
    slicer.mrmlScene.Clear(0)
    
    # get patient name and dice
    patName = row['Name']
    dice = row['Dice']
    
    # print dice and name
    print('Patient: %s' % patName)
    print('Dice: %.5f \n' % dice)
    
    # load nifiti volumes
    ct = slicer.util.loadVolume('../hecktor/data_train/hecktor_nii/' + patName + '/' + patName + '_ct.nii.gz')
    pt = slicer.util.loadVolume('../hecktor/data_train/hecktor_nii/' + patName + '/' + patName + '_pt.nii.gz')

    # segmentations
    gtv_pred = slicer.util.loadLabelVolume('../training_gtvs_noSRM/' + patName + '.nii.gz', properties = {'name': 'PRED'})
    gtv_gt = slicer.util.loadLabelVolume('../hecktor/data_train/hecktor_nii/' + patName + '/' + patName + '_ct_gtvt.nii.gz', properties = {'name': 'GT'})

    # get size
    ct.GetImageData().GetDimensions()
    gtv_pred.GetImageData().GetDimensions()
    gtv_gt.GetImageData().GetDimensions()

    # Convert LabelMap into segmentation node
    gtvSegNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(gtv_pred, gtvSegNode)
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(gtv_gt, gtvSegNode)
    
    # create new separate segmentation nodes that isolate gt and pred segments
    gtvSegNode_gt = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(gtv_gt, gtvSegNode_gt)
    
    gtvSegNode_pred = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(gtv_pred, gtvSegNode_pred)

    # change label map color of heart to red 
    segmentation = gtvSegNode.GetSegmentation()
    segment = segmentation.GetSegment('PRED')
    segment.SetColor(0,0,1)
    segment = segmentation.GetSegment('GT')
    segment.SetColor(1,0,0)

    # turn off fill and increase outline thickness
    segmentDisplayNode = gtvSegNode.GetDisplayNode()
    segmentDisplayNode.SetVisibility2DFill(False)
    segmentDisplayNode.SetSliceIntersectionThickness(4)

    # create 3d view of contours 
    gtvSegNode.CreateClosedSurfaceRepresentation()

    # Select CT as background and turn off foreground and labelmaps
    slicer.util.setSliceViewerLayers(background=ct, foreground=pt, label=None)

    # set windowing of CT
    ctDisplayNode = ct.GetDisplayNode()
    ctDisplayNode.AutoWindowLevelOff()
    ctDisplayNode.SetWindow(350)
    ctDisplayNode.SetLevel(40)

    # set pet colormap and opacity
    ptDisplay = pt.GetDisplayNode()
    PETnodeColor = slicer.util.getFirstNodeByName('PET-Heat')
    ptDisplay.SetAndObserveColorNodeID(PETnodeColor.GetID())
    slicer.util.setSliceViewerLayers(foregroundOpacity=0.3)

    # Compute centroids
    segStatLogic = SegmentStatistics.SegmentStatisticsLogic()
    segStatLogic.getParameterNode().SetParameter("Segmentation", gtvSegNode.GetID())
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.centroid_ras.enabled", str(True))
    segStatLogic.computeStatistics()
    stats = segStatLogic.getStatistics()

    # get centroid
    gt_centroid_ras = stats['GT',"LabelmapSegmentStatisticsPlugin.centroid_ras"]
        
    # Get segmentation as labelmap volume node
    labelMapCtNode_gt = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelMapPtNode_gt = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    
    labelMapCtNode_pred = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelMapPtNode_pred = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')

    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(gtvSegNode_pred, labelMapCtNode_pred, ct)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(gtvSegNode_pred, labelMapPtNode_pred, pt)
    
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(gtvSegNode_gt, labelMapCtNode_gt, ct)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(gtvSegNode_gt, labelMapPtNode_gt, pt)

    # labelmap values based on segmentation order - both are same now
    gt_val = 1
    pred_val = 1 

    # Extract all voxels of CT within the segment as numpy array
    volumeArray = slicer.util.arrayFromVolume(ct)
    labelArray = slicer.util.arrayFromVolume(labelMapCtNode_gt)
    ctIntens_gt[patName] = volumeArray[labelArray==gt_val]
    labelArray = slicer.util.arrayFromVolume(labelMapCtNode_pred)
    ctIntens_pred[patName] = volumeArray[labelArray==pred_val]

    # Extract all voxels of PET within the segment as numpy array
    volumeArray = slicer.util.arrayFromVolume(pt)
    labelArray = slicer.util.arrayFromVolume(labelMapPtNode_gt)
    ptIntens_gt[patName] = volumeArray[labelArray==gt_val]
    labelArray = slicer.util.arrayFromVolume(labelMapPtNode_pred)
    ptIntens_pred[patName] = volumeArray[labelArray==pred_val]
    
    # store volumes of GT and PRED 
    vol_gt[patName] = stats['GT',"LabelmapSegmentStatisticsPlugin.volume_cm3"]
    vol_pred[patName] = stats['PRED',"LabelmapSegmentStatisticsPlugin.volume_cm3"]    

    # remove misc volumes and seg nodes now
    slicer.mrmlScene.RemoveNode(gtv_gt)
    slicer.mrmlScene.RemoveNode(gtv_pred)    
    slicer.mrmlScene.RemoveNode(gtvSegNode_gt)
    slicer.mrmlScene.RemoveNode(gtvSegNode_pred)
    
    # move slices to segment centroid
    slicer.modules.markups.logic().JumpSlicesToLocation(gt_centroid_ras[0], gt_centroid_ras[1], gt_centroid_ras[2], True)

    # zoom in 30% more - axial
    zoom = 0.3
    layoutManager = slicer.app.layoutManager()
    red = layoutManager.sliceWidget('Red')
    redLogic = red.sliceLogic()
    x,y,z = redLogic.GetSliceNode().GetFieldOfView()
    redLogic.GetSliceNode().SetFieldOfView(x*zoom,y*zoom,y*zoom)
    
    # sagittal zoom
    green = layoutManager.sliceWidget('Green')
    greenLogic = green.sliceLogic()
    x,y,z = greenLogic.GetSliceNode().GetFieldOfView()
    greenLogic.GetSliceNode().SetFieldOfView(x*zoom,y*zoom,y*zoom)
    
    # coronal zoom
    yellow = layoutManager.sliceWidget('Yellow')
    yellowLogic = yellow.sliceLogic()
    x,y,z = yellowLogic.GetSliceNode().GetFieldOfView()
    yellowLogic.GetSliceNode().SetFieldOfView(x*zoom,y*zoom,y*zoom)

    # display orthogonal slices in all planes
    display(slicernb.ViewSliceDisplay('Red'), slicernb.ViewSliceDisplay('Yellow'), slicernb.ViewSliceDisplay('Green'))

    # optimize view of the 3D window
    viewNode = slicer.app.layoutManager().threeDWidget(0).mrmlViewNode()
    viewNode.SetBackgroundColor(0,0,0)
    viewNode.SetBackgroundColor2(0,0,0)
    viewNode.SetAxisLabelsVisible(False)
    viewNode.SetBoxVisible(False)

    # zoom in on the tumor
    camera = slicer.mrmlScene.GetFirstNodeByClass('vtkMRMLCameraNode').GetCamera()
    camera.Dolly(5)

    # center the 3D View on the scene
    layoutManager = slicer.app.layoutManager()
    threeDWidget = layoutManager.threeDWidget(0)
    threeDView = threeDWidget.threeDView()
    threeDView.resetFocalPoint()

    # rotate the 3D view
    threeDView.pitch()

    # display 3D ground truth and autoseg
    display(slicernb.View3DWidget())

    # Show light box view 
    display(slicernb.ViewLightboxDisplay(viewName='Red', filename=patName + '_axial.png', rows=5, columns=5, positionRange=[gt_centroid_ras[2]-25, gt_centroid_ras[2]+25])) 
    display(slicernb.ViewLightboxDisplay(viewName='Green', filename=patName + '_coronal.png', rows=5, columns=5, positionRange=[gt_centroid_ras[1]-25, gt_centroid_ras[1]+25])) 
    display(slicernb.ViewLightboxDisplay(viewName='Yellow', filename=patName + '_sagittal.png', rows=5, columns=5, positionRange=[gt_centroid_ras[0]-25, gt_centroid_ras[0]+25])) 
    
    # increment i
    i += 1

In [ ]:
# add segmentation statistics to dataframe
df = pd.read_csv('../nnU-NetResultsTrain.csv') # load orig
df = df.set_index('Name')

# gather stats as list
segStatsList = (ctIntens_gt, ptIntens_gt, vol_gt, ctIntens_pred, ptIntens_pred, vol_pred)
cols = {0: 'ctIntens_GT', 1: 'ptIntens_GT',2: 'vol_GT',3: 'ctIntens_PRED',4: 'ptIntens_PRED',5: 'vol_PRED'}

# build segs df
df_segs = pd.DataFrame(segStatsList)
df_segs = df_segs.transpose()
df_segs = df_segs.rename(columns=cols)

# merge df 
df_all = df.join(df_segs)

In [ ]:
# save data frame 
pd.to_pickle(df_all, '../dataAll.pkl')

In [ ]:
# write csv with mean values instead
ctIntens_gt_mean = {k: np.nanmean(v) for k,v in ctIntens_gt.items()}
ptIntens_gt_mean = {k: np.nanmean(v) for k,v in ptIntens_gt.items()}

ctIntens_pred_mean = {k: np.nanmean(v) for k,v in ctIntens_pred.items()}
ptIntens_pred_mean = {k: np.nanmean(v) for k,v in ptIntens_pred.items()}

# gather stats as list
segStatsList_mean = (ctIntens_gt_mean, ptIntens_gt_mean, vol_gt, ctIntens_pred_mean, ptIntens_pred_mean, vol_pred)

# build segs df
df_segs_mean = pd.DataFrame(segStatsList_mean)
df_segs_mean = df_segs_mean.transpose()
df_segs_mean = df_segs_mean.rename(columns=cols)

# merge df 
df_all_mean = df.join(df_segs_mean)

In [ ]:
df_all_mean

In [ ]:
df_all_mean.to_csv('../dataAllMean.csv')